# Lua Tutorial

# Contents

1.) Motivation behind using Lua <br>
2.) Variables and data structures <br>
3.) Basic control-flow structures <br>
4.) Object-oriented programming <br>
5.) Torch tensors and storage <br>
6.) Basic Tensor operations <br>
7.) Calling C functions from lua using ffi library 

## Motivation behind using Lua

1.) LuaJit is a faster interpreter. A comparitive study on Fibonaaci 40th number in Python and Lua showed Lua was 20 times faster over Python. 
Execution time in Lua : 2 second
Execution tim in Python : 37.28 seconds

In [9]:
function fib(n)
    if n<2 then return 1 end
    return fib(n-2) + fib(n-1)
end
function fibOuter(n)
    start_time = os.time()
    print(fib(n))
    end_time = os.time()
    print('start time: '   .. start_time .. 's')
    print('end time: '     .. end_time .. 's')
    elapsed_time = os.difftime(end_time,start_time)
    print('time elapsed: ' .. elapsed_time .. 's')
end
fibOuter(40)

165580141	
start time: 1462759362s	
end time: 1462759364s	
time elapsed: 2s	


2.) Lua has a smaller memory footprint. On the above example memory profiling showed that Lua took 2 times less memory Python
Memory footprint in Lua : 927 bytes
Memory footprint in Python : 2136 bytes

Here are some benchmark performance numbers for few examples http://benchmarksgame.alioth.debian.org/u64q/compare.php?lang=lua&lang2=python3

3.) Closures are a very powerful tool in Lua, other scripting languages like Python support anonymous and lambda functions but not closures in their entirety. In Python, the enclosing state variable are read-only.

In [4]:
function generator()
    local i = 0
    return function()
        if i>=5 then return nil 
        else 
            i=i+1
            return i
        end
    end
end

for val in generator() do
    print(val)
end

1	
2	
3	
4	
5	


4.) Light-weight, can be used for embedded systems. For example, the size of python22.dll is 824kb, however, basic vanilla Lua engine, including parser/compiler/interpreter weighs under 100kb.

5.) The FFI interface is very simple to implement any C/C++ extensions

## Variables and Data Structures

In [9]:
local value = 26 --By default number types are double types
str = 'immutable' --Strings are immutable so avoid concatenation
-- Instead use string format
print(string.format('%s %d',str,value))

immutable 26	


In [10]:
-- Lua has garbage collector anything set to nil will be collected
str = nil
collectgarbage()

nil	


#### Lua's basic datastructure are associative arrays, tables. We can store heterogenous type objects in same table.

In [11]:
--create a table
local t = {}
table.insert(t,'1')
t['2']='two'
t.three = '3'
print("printing table object..")
print(t)
print("scanning over table..")
for id, value in pairs(t) do
    print(string.format("%s %s", id, value))
end
print("access table indices..")
print(string.format("t[1] : %s",t[1]))
print(string.format("t[2] : %s",t[2])) -- nil because key is string
print(string.format("t['2'] : %s",t['2']))
print(string.format("t['2'] : %s",t['three']))

--json style way of defining table
t = {one = '1', two = '2'}
print("printing json-style table format..")
print(t)
print(string.format("t.one : %s",t.one)) -- like a member variable

printing table object..	
{
  1 : 1
  2 : two
  three : 3
}
scanning over table..	
1 1	
2 two	
three 3	
access table indices..	
t[1] : 1	
t[2] : nil	
t['2'] : two	
t['2'] : 3	
printing json-style table format..	
{
  one : 1
  two : 2
}
t.one : 1	


In [1]:
--tables as lists
local t = {'one','two','three'}
print(t)

{
  1 : one
  2 : two
  3 : three
}


## Basic control-flow structures

In [11]:
-- while construct
local count=0
while count<10 do
    count=count+1
    print(count)
end

1	
2	
3	
4	
5	
6	
7	
8	
9	
10	


In [2]:
-- if-else clause
local value = 100
if value <= 5 then print("Value is between 0-5") 
elseif value <= 10 then print("Value is between 5-10")
else print("Value is greater than 10")
end

Value is greater than 10	


In [4]:
-- for construct
local sum = 0
for i=1, 10 do --lower and upper bounds included
  sum = sum+i
end
print(string.format("sum: %s",sum))

sum: 55	


In [6]:
-- higher-order functions or closures
function hypotenuse(x)
    return function(y) return x*x + y*y end
end

h=hypotenuse(3)
print(h(4))

25	


In [7]:
-- function iterators
list = {10,'eleven',20,'twenty-one',30,'thirty-one'}
function iterate (t)
  local i = 0
  local n = table.getn(t)
  return function ()
         i = i + 1
         if i <= n then return t[i] end
         end
end

for val in iterate(list) do
    print(val)
end

10	
eleven	
20	
twenty-one	
30	
thirty-one	


In [69]:
-- co-routines
local iterator = coroutine.create(function ()
           for i=1,3 do
             print("iteration : ", i)
             coroutine.yield()
           end
         end)

print(coroutine.status(iterator))
print(coroutine.resume(iterator))
print(coroutine.status(iterator))
print(coroutine.resume(iterator))
print(coroutine.status(iterator))
print(coroutine.resume(iterator))
print(coroutine.status(iterator))
print(coroutine.resume(iterator))
print(coroutine.status(iterator))
print(coroutine.resume(iterator))

suspended	
iteration : 	1	
true	
suspended	
iteration : 	2	
true	
suspended	
iteration : 	3	
true	
suspended	
true	
dead	
false	cannot resume dead coroutine	


### How are coroutines different from python generators?

1.) Python yield is a syntactic sugar which can only be called from within the body of the generator <br>
2.) Lua coroutines are asymmetric which means there is one function to resume a suspended co-routine and a different function to suspend a coroutine

## Object-Oriented paradigm

In [10]:
t = {} --regular table
mt = {} --metatable, empty for now
setmetatable(t, mt)
print(getmetatable(t)) --returns mt

{
}


In [35]:
-- __index metamethod
-- whenever a table is looked up, for eg, t[1], t['one'], t.one
print("table as metatable")
local mt = {
  __index = {x = 0, y = 0, w = 100, h = 100}
}
setmetatable(t, mt)
print(t.x)
print(t['w'])
print(t.z)

print("function as metatable")
--metamethods can be tables or functions
local mt = {
  __index = function(t, key)
              if key == 'z' then return 100 
              else return table[key] end
            end
}
setmetatable(t, mt)
print(t.x)
print(t['w'])
print(t.z)


table as metatable	
0	
100	
nil	
function as metatable	
nil	
nil	
100	


In [38]:
--[[Optional: <Not used in framework>
    Simple class example -- Taken from https://www.lua.org/pil/13.4.1.html]]

--create a class
Window = {}
--create prototypical keyset with default values
Window.prototype = {x=0, y=0, width=100, height=100 }
Window.mt = {}
--declare the constructor
function Window.new(o)
    setmetatable(o, Window.mt)
    return o
end

Window.mt.__index = function(table, key)
  return Window.prototype[key]
end

w=Window.new({x=10,y=20})
print(w.width)
print(w.x)

w=Window.new({})
print(w.width)
print(w.x)

100	
10	
100	
0	


In [1]:
-- Torch class interface
local torch = require 'torch'

local C1 = torch.class('torch.C1')
function C1:__init() -- equivalent to P.__init(self)
  self.x = 0
  self.y = 1
end
function C1.printMembers(self)
  print(string.format("x=%s y=%s",self.x, self.y))
end

local C2, parent = torch.class('torch.C2','torch.C1')
function C2:__init()
  parent:__init() -- or self.__parent:__init()
  self.w = 100
  self.h = 100
end
function C2.printMembers(self)
  print(string.format("x=%s y=%s",self.x, self.y, ))
end

print ("Base class")
local c1 = torch.C1()
c1:printMembers()      -- function call 1
c1.printMembers(c1)    -- function call 2

print ("Inherited class")
local c2 = torch.C2()
print(c2.x)
print(c2.w)
c2:printMembers()

Base class	
x=0 y=1	
x=0 y=1	
Inherited class	
0	
100	
x=0 y=1	


## Torch tensors and storage 

1.) N-dimensional arrays which are views over an underlying storage <br>
2.) Different tensors can refer to same storage <br>
3.) Different Types of tensors : FloatTensor, DoubleTensor, CudaTensor <br>

In [7]:
a = torch.DoubleTensor(3,4)
print(a)             --garbage content
a:fill(1)
print(a)             --set all values to 1 

-3.1050e+231 -3.1050e+231  6.4229e-323   0.0000e+00
  3.3581e+30  1.1286e+277  2.8751e+161  1.4776e+248
  1.1610e-28  6.2499e-143   1.1610e-28   1.6889e-52
[torch.DoubleTensor of size 3x4]

 1  1  1  1
 1  1  1  1
 1  1  1  1
[torch.DoubleTensor of size 3x4]



In [8]:
-- indexing and slicing
print(a[{1,1}])

1	



In [13]:
--underlying storage
a = torch.rand(3,4) --uniform samples between (0,1)
b = a:transpose(1,2) -- transpose between dimension 1 and 2
print(a:storage())
print(b:storage())
print(string.format("a=%s",a))
print(string.format("b=%s",b))

 0.2508
 0.4766
 0.2709
 0.6824
 0.8910
 0.6375
 0.8182
 0.6139
 0.0232
 0.2868
 0.4462
 0.2112
[torch.DoubleStorage of size 12]

 0.2508
 0.4766
 0.2709
 0.6824
 0.8910
 0.6375
 0.8182
 0.6139
 0.0232
 0.2868
 0.4462
 0.2112
[torch.DoubleStorage of size 12]

a= 0.2508  0.4766  0.2709  0.6824
 0.8910  0.6375  0.8182  0.6139
 0.0232  0.2868  0.4462  0.2112
[torch.DoubleTensor of size 3x4]
	
b= 0.2508  0.8910  0.0232
 0.4766  0.6375  0.2868
 0.2709  0.8182  0.4462
 0.6824  0.6139  0.2112
[torch.DoubleTensor of size 4x3]
	


In [14]:
print(a:isContiguous())
print(b:isContiguous()) -- b holds reference to indices in a, doesn't create a new storage
print(string.format("a[{1,1}]=%s",a[{1,1}]))
print(string.format("a[{1,2}]=%s",a[{1,2}])) 
print(string.format("a[{2,1}]=%s",a[{2,1}]))
print(string.format("b[{1,1}]=%s",b[{1,1}])) 
print(string.format("b[{1,2}]=%s",b[{1,2}])) 
print(string.format("b[{2,1}]=%s",b[{2,1}]))

true	
false	
a[{1,1}]=0.25083295046352	
a[{1,2}]=0.47656729188748	
a[{2,1}]=0.89103118982166	
b[{1,1}]=0.25083295046352	
b[{1,2}]=0.89103118982166	
b[{2,1}]=0.47656729188748	


In [16]:
c=b:clone()   --allocates new memory
print(b)
print(c:isContiguous())

 0.2508  0.8910  0.0232
 0.4766  0.6375  0.2868
 0.2709  0.8182  0.4462
 0.6824  0.6139  0.2112
[torch.DoubleTensor of size 4x3]

true	


In [17]:
print(string.format("c[{1,1}]=%s",c[{1,1}]))
print(string.format("c[{1,2}]=%s",c[{1,2}]))
print(string.format("c[{2,1}]=%s",c[{2,1}]))

c[{1,1}]=0.25083295046352	
c[{1,2}]=0.89103118982166	
c[{2,1}]=0.47656729188748	


## Basic Tensor operations

### Scalar operations

In [46]:
local mat = torch.FloatTensor(2,3):fill(2)
print("mat = "); print(mat)
print("Adding constant 3 to mat = "); print(torch.add(mat, 3))
print("Multiply constant 3 to mat = "); print(torch.mul(mat, 3))

mat = 	
 2  2  2
 2  2  2
[torch.FloatTensor of size 2x3]

Adding constant 3 to mat = 	
 5  5  5
 5  5  5
[torch.FloatTensor of size 2x3]

Multiply constant 3 to mat = 	
 6  6  6
 6  6  6
[torch.FloatTensor of size 2x3]



### Element-wise operations

In [48]:
local mat = torch.FloatTensor(2,3):fill(2)
print("mat = "); print(mat)
-- add two 2D matrix
print("Add a 2D tensor to mat = ")
print(torch.add(mat, torch.FloatTensor(2,3):fill(1.5)))
-- add a vector to 2D matrix
print("Add a flat tensor to mat = ")
print(torch.add(mat, torch.FloatTensor(6):fill(2.5)))
-- multiply two 2D matrix
print("Multipy a tensor to a mat")
print(torch.cmul(mat, torch.FloatTensor(2,3):fill(1.5)))

mat = 	
 2  2  2
 2  2  2
[torch.FloatTensor of size 2x3]

Add a 2D tensor to mat = 	
 3.5000  3.5000  3.5000
 3.5000  3.5000  3.5000
[torch.FloatTensor of size 2x3]

Add a flat tensor to mat = 	
 4.5000  4.5000  4.5000
 4.5000  4.5000  4.5000
[torch.FloatTensor of size 2x3]

Multipy a tensor to a mat	
 3  3  3
 3  3  3
[torch.FloatTensor of size 2x3]



### Matrix operations

1.) Matrix Multiplication <br>
[res]torch.addmm([res,][beta,] [v1,] M [v2,] mat1, mat2)

res = (res X beta) + (v1 X M) + (v2 X mat1 X mat2)

2.) Matrix-vector multiplication <br>
[res] torch.addmv([res,] [beta,] [v1,] vec1, [v2,] mat, vec2)

res = beta X res + v1 X vec1 + v2 X mat X vec2

In [65]:
print("Matrix-vector multiplication\n")
local mat1 = torch.FloatTensor(2):fill(2)
print("mat1 = ")
print(mat1)
local mat2 = torch.FloatTensor(3,2):fill(3)
print("mat2 = ")
print(mat2)
local output = torch.FloatTensor(3):fill(0)
output:addmv(mat2, mat1)
print(output)

print("Matrix-matrix multiplication\n")
local mat1 = torch.FloatTensor(2,3):fill(2)
print("mat1 = ")
print(mat1)
local mat2 = torch.FloatTensor(3,2):fill(3)
print("mat2 = ")
print(mat2)
local output = torch.FloatTensor(2,2):fill(0)
output:addmm(mat1, mat2)
print(output)

Matrix-vector multiplication
	
mat1 = 	
 2
 2
[torch.FloatTensor of size 2]

mat2 = 	
 3  3
 3  3
 3  3
[torch.FloatTensor of size 3x2]

 12
 12
 12
[torch.FloatTensor of size 3]

Matrix-matrix multiplication
	
mat1 = 	
 2  2  2
 2  2  2
[torch.FloatTensor of size 2x3]

mat2 = 	
 3  3
 3  3
 3  3
[torch.FloatTensor of size 3x2]

 18  18
 18  18
[torch.FloatTensor of size 2x2]



## Calling C functions from lua using ffi library 

1.) This is one of the easiest ways to call external C routines through lua and gets rid of the lua bindings from C. <br>
2.) The other way to do that is by using built-in lua bindings in C code to add new functionalities. For further details refer to https://www.lua.org/pil/24.1.html

In [ ]:
local ffi = require("ffi")
ffi.cdef[[
typedef struct { double x, y; } point_t;
]]

local point
local mt = {
  __add = function(a, b) return point(a.x+b.x, a.y+b.y) end,
  __len = function(a) return math.sqrt(a.x*a.x + a.y*a.y) end,
  __index = {
    area = function(a) return a.x*a.x + a.y*a.y end,
  },
}
point = ffi.metatype("point_t", mt)

local a = point(3, 4)
print(a.x, a.y)  
print(#a)        
print(a:area())  
local b = a + point(0.5, 8)
print(#b)